In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from pyts.image import GramianAngularField

from sklearn.preprocessing import StandardScaler
import datetime

import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras import layers
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer
from keras.optimizers import Adam

In [2]:
from keras.models import load_model

model = load_model('Model6.h5')

In [3]:
def generate_GAF(data):
    feature_names = ["RAIN FALL CUM. SINCE 0300 UTC (mm)","TEMP. ('C)","RH (%)","WIND SPEED 10 m (Kt)","SLP (hPa)","MSLP (hPa / gpm)"]

    gaf_images = []
    for feature_name in feature_names:
        # Get the data for the current feature
        data_feature = data[0][feature_name]
        # Fill NaN values with the mean of the column
        data_feature.fillna(data_feature.mean(), inplace=True)
        # Convert the data to a numpy array
        data_feature = data_feature.values
        # Create a Gramian Angular Field object
        gaf = GramianAngularField(image_size=256, method='summation')
        # Convert the data to a GAF image
        image = gaf.transform([data_feature])
        gaf_images.append(image[0])
    return gaf_images

In [ ]:
current_date = datetime.date.today() #Current Date
prev_date = current_date - datetime.timedelta(days=6) #Date 6 Days before
# Get the Location Information
state = "UTTARAKHAND"
district = input("Enter the district: ") #DEHRADUN
station = input("Enter the station: ") #MUSSOORIE(UKG)_UKG
#Get the Data from IMD
url = "http://aws.imd.gov.in:8091/AWS/dataview.php?a=AWS&b={}&c={}&d={}&e={}&f={}&g=ALL_HOUR&h=ALL_MINUTE".format(state,district,station,prev_date, current_date)
df_list = pd.read_html(url)
#Generate the GAF
gaf_images = generate_GAF(df_list)
gaf_data = np.array([gaf_images])
#Loading the 6 Hour Model and make prediction
from keras.models import load_model
model = load_model('Model6.h5')
prediction = model.predict(gaf_data)
print("probability of cloudburst :- "+ (str)((prediction*100)[0][0]) +"%")